In [ ]:
#Description: I was not able to find a better h w.r.t the third proposed at lesson
#to improve performance I included from beginning the mandatory set (those who present a tile that is not present in other)
#and also forced an ordering to avoid exploring same node in a different permutation

In [1]:
from random import random
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue, SimpleQueue, LifoQueue
import heapq as hq

from itertools import chain

#import networkx as nx
#from networkx.drawing.nx_pydot import graphviz_layout
#import matplotlib.pyplot as plt

import numpy as np
import math


In [20]:
PROBLEM_SIZE = 20 #5,10,15,20,50,100,500,1000
NUM_SETS = 40 #10,20,30,40,100,200,1000,2000
P_TILE = 0.1 #0.3,0.2,0.15,0.1,0.05,0.03,0.05,0.03

SETS = tuple(np.array([random() < P_TILE for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))
State = namedtuple('State', ['taken', 'not_taken'])

def goal_check(state):
    return np.all(reduce(np.logical_or, [SETS[i] for i in state.taken], np.array([False for _ in range(PROBLEM_SIZE)])))


assert goal_check(State([i for i in range(NUM_SETS)], [])), "Probelm not solvable"

#SOURCE = "0,0F"
#sspace = nx.Graph()

print(SETS)

def node_repr(count,state,isdest):
    if isdest:
        return f"{count},{len(state.taken)}T"
    return f"{count},{len(state.taken)}F"

(array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False,  True, False, False, False,
       False, False]), array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False]), array([ True, False, False,  True, False, False, False, False, False,
       False,  True, False,  True, False, False, False, False, False,
       False, False]), array([False, False, False,  True, False, False, False, False, False,
       False,  True, False, False, False,  True, False, False, False,
       False, False]), array([ True, False, False, False, False, False, False,  True, False,
       False, False, False,  True, False, False, False,  True, False,
        True, False]), array([False, False, False, False,  True, False, False, False, False,
       False, False, False, False,  True, False, False,  True,  True,
       False, False]), array([False, False, 

In [21]:
#greedy
def get_priority(state):
    set_covered=reduce(np.logical_or, [SETS[i] for i in state.taken], np.array([False for _ in range(PROBLEM_SIZE)]))
    return PROBLEM_SIZE-np.count_nonzero(set_covered)

frontier = PriorityQueue()
initial_state = State([], [i for i in range(NUM_SETS)])
frontier.put((0,initial_state))

counter=0
current_state = frontier.get()[1]
priority = PROBLEM_SIZE

while priority > 0:
    counter += 1
    for i,action in enumerate(current_state.not_taken):
        
        new_taken = current_state.taken.copy()
        new_taken.append(action)
        new_not_taken = current_state.not_taken.copy()
        new_not_taken.pop(i)
        
        new_state = State(new_taken, new_not_taken)

        frontier.put((get_priority(new_state),new_state)) 
    
    priority_and_state = frontier.get()
    current_state=priority_and_state[1]
    priority=priority_and_state[0]

print(current_state)
print(len(current_state.taken))
print(f"Solved in {counter:,} steps")
print(goal_check(current_state))

greedy_best = len(current_state.taken)


State(taken=[4, 13, 3, 5, 8, 9, 12], not_taken=[0, 1, 2, 6, 7, 10, 11, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39])
7
Solved in 7 steps
True


In [23]:
#"BFS" order+necessary sets #comparison algorithm
#idea from @Big_Pan on telegram group
#improve by forcing priority of list of set that do not contain essentials set to +INF

#computing essentials set

necessary_sets=[]

for n,set in enumerate(SETS):
    NEW_SETS=tuple(SETS[i] for i in chain(range(n),range(n+1,NUM_SETS)))

    other_set_covered=reduce(np.logical_or, [NEW_SETS[i] for i in range(NUM_SETS-1)] , np.array([False for _ in range(PROBLEM_SIZE)]))
    set_still_not_covered=np.where(other_set_covered,False,True)
    #set_complete=np.all(reduce(np.logical_or,set_still_not_covered,set)) # optional if we check that goal()==true is sufficient np.any(set_still_not_covered)
    set_complete=np.any(set_still_not_covered)
    if set_complete:
        necessary_sets.append(n)

print(f"necessary set:{necessary_sets}")

#those set are included by default and doesn't count in ordering check

def is_ordered(state):
    last_set=len(necessary_sets)
    for set in state.taken[last_set:]:
        if set<last_set:
            return False
        last_set=set
    return True

frontier = SimpleQueue()
initial_state = State([], [i for i in range(NUM_SETS)])

#updating initial state
for s in necessary_sets:
    initial_state[0].append(s)
    initial_state[1].remove(s)

frontier.put(initial_state)

counter = 0
current_state = frontier.get()

while not goal_check(current_state):
    #sspace.add_node(node_repr(counter,current_state,goal_check(current_state))) #too much edges
    
    counter += 1
    
    for i,action in enumerate(current_state.not_taken):
        
        new_taken = current_state.taken.copy()
        new_taken.append(action)
        new_not_taken = current_state.not_taken.copy()
        new_not_taken.pop(i)
        
        new_state = State(new_taken, new_not_taken)

        if is_ordered(new_state):
            frontier.put(new_state)

    if (counter % 10000)== 0:       
        print(len(current_state.taken))#just to check progress

    current_state=frontier.get()     



BFS_best=len(current_state.taken)
print(current_state.taken)
print(f"len={len(current_state.taken)}")
print(f"Solved in {counter:,} steps")


necessary set:[12]
5
5
5
5
5
5
5
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
[12, 3, 4, 5, 8, 9, 13]
len=7
Solved in 1,398,787 steps


In [31]:
#A* original

#has to be optimistic evaluation, is not admissible
def h(state):
    set_covered=reduce(np.logical_or, [SETS[i] for i in state.taken], np.array([False for _ in range(PROBLEM_SIZE)]))
    return PROBLEM_SIZE-np.count_nonzero(set_covered)

def covered(state):
    return reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    )

def h3(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    missing_size = PROBLEM_SIZE - sum(already_covered)
    candidates = sorted((sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS), reverse=True)
    taken = 1
    while sum(candidates[:taken]) < missing_size:
        taken += 1
    return taken

#not admissible
def h4(state):
    
    already_covered = covered(state)
    not_covered =  np.logical_not(already_covered)
    if np.all(already_covered):
        return 0
    missing_size = PROBLEM_SIZE - sum(already_covered)

    covered_by_not_taken_set=[np.logical_and(SETS[i],not_covered) for i in state.not_taken]
    candidates = sorted([(sum(s),s) for s in covered_by_not_taken_set], reverse=True , key=lambda a: a[0])
    #print(candidates)
    taken = 1
    while not np.all(reduce(np.logical_or,[candidates[i][1] for i in range(taken)],already_covered)):
        taken += 1 #no: can overestimate the cost
        #4 example it if you have [(4,...),(4,...),(4,...),(3,...)] and you have to cover 8 you can cover it with 2
        #but in this case if only the third allows you complete it doesn't mean that you have to take already 3 set maybe chosing just 1 and 3 is sufficient
    return taken




frontier = PriorityQueue()
initial_state = State([], [i for i in range(NUM_SETS)])
frontier.put((h3(initial_state),(initial_state,h3(initial_state))))

counter=0
current_state = frontier.get()[1][0]
exp_cost = PROBLEM_SIZE

while exp_cost > 0:
    counter += 1
    for i,action in enumerate(current_state.not_taken):
        
        new_taken = current_state.taken.copy()
        new_taken.append(action)
        new_not_taken = current_state.not_taken.copy()
        new_not_taken.pop(i)
        
        new_state = State(new_taken, new_not_taken)
        g=len(new_state.taken)
        exp_cost_new=h3(new_state)
        frontier.put((g+exp_cost_new,(new_state,exp_cost_new))) 
    
    priority_and_state = frontier.get()[1]
    current_state=priority_and_state[0]
    exp_cost=priority_and_state[1]

print(current_state)
print(len(current_state.taken))
print(f"Solved in {counter:,} steps")
print(goal_check(current_state))

greedy_best = len(current_state.taken)

#not executed too slow

In [33]:

#improved ordered A* that accounts for ordering and essentials sets

#computing essentials set
#idea from @Big_Pan on telegram group
#improve by forcing priority of list of set that do not contain essentials set to +INF

#other heuristic can be the importance of the set in terms of how much its tiles are "rare"
sum_val=reduce(np.add, [np.where(SETS[i],1,0) for i in range(NUM_SETS)], np.array([0 for _ in range(PROBLEM_SIZE)]))
rarity=1/sum_val
MAX_COST=np.sum(rarity)


def calculate_rarity(state):
    
    #MAX_COST=np.sum(rarity)
    set_covered=reduce(np.logical_or, [SETS[i] for i in state.taken], np.array([False for _ in range(PROBLEM_SIZE)]))
    return MAX_COST-np.sum(np.where(set_covered,1,0)*rarity) #cost in rerm of rarity of the remaining sets
    #never overestimates if we use an appropriate starting point, monotonic
    #1/2 1
    #we choose 1
    #at this point 
    #estimated_cost=2/1.5*(2-1)=0.5/1.5*2=1/3 < 1
    #we choose 2
    #estimated_cost=2/1.5*(2-0.5)= 2 > 1

def g(state):
    return len(state.taken)

def dist(state):#incorrect (pessimistic estimate)
    set_covered=reduce(np.logical_or, [SETS[i] for i in state.taken], np.array([False for _ in range(PROBLEM_SIZE)]))
    return PROBLEM_SIZE-np.count_nonzero(set_covered)

def covered(state):
    return reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    )


def h3(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    missing_size = PROBLEM_SIZE - sum(already_covered)
    candidates = sorted((sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS), reverse=True)
    taken = 1
    while sum(candidates[:taken]) < missing_size:
        taken += 1
    return taken


def h4(state):
    
    already_covered = covered(state)
    not_covered =  np.logical_not(already_covered)
    if np.all(already_covered):
        return 0
    missing_size = PROBLEM_SIZE - sum(already_covered)

    not_covered_by_s=[np.logical_and(SETS[i],not_covered) for i in state.not_taken]
    candidates = sorted([(sum(s),s) for s in not_covered_by_s], reverse=True , key=lambda a: a[0])
    #print(candidates)
    taken = 1
    while not np.all(reduce(np.logical_or,[candidates[i][1] for i in range(taken)],already_covered)):
        taken += 1
        #not admissible since it can overestimate the cost
    return taken

#the idea to use past distance came up to my mind 
#both reading the lab suggestion
#and watching https://en.wikipedia.org/wiki/A*_search_algorithm
#in the animation that shows A* solving a path in American railroad network

#h not admissible but works well
def new_h(past_dist, curr_dist):
    return curr_dist-(curr_dist-past_dist)**2 
    #it is more free to explore but no more monotonically decreasing,
    #so not more straight to the correct direction
    #curr_dist - (curr_dist-past_dist)**2
    #if last step added 2
    # result= b+2-4 =b-2
    # now add 0
    # result = b+2-0 =b+2
    # now add 2
    # result = b+2+2-4 = b so no monotonic
    # 
    # curr_dist/2 + past_dist/2
    # curr_dist = 2b+4
    # (b+2)+b
    # -> 2b+2 -> no monotonic
    #                        
    #the concept is to decrease distance of solution that are "similar" to the greedy best

# curr_dist<past_dist for sure
# admissible
# if curr_dist nearer to solution curr_dist-past_dist<0 --> increase priority

#idea from @Big_Pan on telegram group
#improve by forcing priority of list of set that do not contain essentials set to +INF

#computing essentials set

necessary_sets=[]

for n,set in enumerate(SETS):
    NEW_SETS=tuple(SETS[i] for i in chain(range(n),range(n+1,NUM_SETS)))

    other_set_covered=reduce(np.logical_or, [NEW_SETS[i] for i in range(NUM_SETS-1)] , np.array([False for _ in range(PROBLEM_SIZE)]))
    set_still_not_covered=np.where(other_set_covered,False,True)
    #set_complete=np.all(reduce(np.logical_or,set_still_not_covered,set)) # optional if we check that goal()==true is sufficient np.any(set_still_not_covered)
    set_complete=np.any(set_still_not_covered)
    if set_complete:
        necessary_sets.append(n)

print(f"necessary set:{necessary_sets}")

#those set are included by default and doesn't count in ordering check

def is_ordered(state):
    last_set=len(necessary_sets)
    for set in state.taken[last_set:]:
        if set<last_set:
            return False
        last_set=set
    return True

frontier = PriorityQueue()
initial_state = State([], [i for i in range(NUM_SETS)])

#updating initial state
for s in necessary_sets:
    initial_state[0].append(s)
    initial_state[1].remove(s)

#0 since first priority doesn't really count
frontier.put((h3(initial_state),(initial_state,h3(initial_state))))


counter=0
current_state = frontier.get()[1][0]
exp_dist_old = PROBLEM_SIZE

while not (exp_dist_old == 0):
    counter += 1
   
    for i,action in enumerate(current_state.not_taken):
        
        new_taken = current_state.taken.copy()
        new_taken.append(action)
        new_not_taken = current_state.not_taken.copy()
        new_not_taken.remove(action)
        
        new_state = State(new_taken, new_not_taken)

        if is_ordered(new_state):
            act_cost_g = g(new_state)
            exp_dist = h3(new_state)
            priority = act_cost_g+exp_dist

            frontier.put((priority, (new_state, exp_dist)))

    priority__state_dist_old = frontier.get()

    current_state_exp_dist_old=priority__state_dist_old[1]

    current_state=current_state_exp_dist_old[0]
    exp_dist_old=current_state_exp_dist_old[1]

    
    #priority=priority_and_state[0] #not necessary



print(current_state)
print(len(current_state.taken))
print(f"Solved in {counter:,} steps")
print(goal_check(current_state))

#ex. improve to 1.2s


necessary set:[12]
State(taken=[12, 3, 4, 5, 8, 9, 13], not_taken=[0, 1, 2, 6, 7, 10, 11, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39])
7
Solved in 2,006 steps
True
